<a href="https://colab.research.google.com/github/Ankit-Kumar-Jha-01/ML-Sentiment-analysis-of-comments-received-through-E-consultation-module./blob/main/Final_Sentimental_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount with Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**To check is there GPU active or not**

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


**Install required packages**

In [ ]:
!pip install datasets torch scikit-learn pandas numpy matplotlib seaborn wordcloud nltk emoji accelerate nlpaug
!pip install wandb --upgrade
!pip uninstall -y transformers
!pip uninstall -y transformers

!pip install --upgrade transformers==4.56.0


import nltk
nltk.download('punkt')
nltk.download('stopwords')

import wandb
wandb.login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 36.5 MB/s eta 0:00:00
Found existing installation: transformers 4.56.0
Uninstalling transformers-4.56.0:
  Successfully uninstalled transformers-4.56.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 131.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kumarankitjha2907 (www-cit-ac-in) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

**Access the CSV file from Drive**

In [ ]:
import pandas as pd
import re
import emoji
from nltk.corpus import stopwords

file_path = "/content/drive/MyDrive/Sentimental_comments/comments_dataset.csv"

df = pd.read_csv(file_path)

df.head()

,id,comment,sentiment
0,1,Portal frequently crashes under Secretarial St...,negative
1,2,Lacks clarity on penalties,negative
2,3,Confusing requirements under Companies Act,negative
3,4,Our company seek clarification on requirements...,neutral
4,5,More paperwork required under Corporate Govern...,negative


**Preprocess the comments :**

remove emoji,
remove special character,
lowercase the statement

In [ ]:
def clean_text(text):
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^a-zA-Z0-9\s!?]', '', text)
    text = text.lower().strip()
    return text

df['clean_comment'] = df['comment'].apply(clean_text)


**Augmentation of statement**:


synonym replacement,
random insertion,
random swap,
random deletion

**Special augmentation for neutral data**

In [ ]:
import pandas as pd
import re
import emoji
import random
import nlpaug.augmenter.word as naw
from nltk.corpus import stopwords
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

syn_aug = naw.SynonymAug(aug_src='wordnet')

def synonym_replacement(text):
    try:
        return syn_aug.augment(text)
    except:
        return text

def random_insertion(text):
    words = text.split()
    if len(words) < 2:
        return text
    insert_pos = random.randint(0, len(words)-1)
    insert_word = random.choice(words)
    words.insert(insert_pos, insert_word)
    return ' '.join(words)

def random_swap(text):
    words = text.split()
    if len(words) < 2:
        return text
    idx1, idx2 = random.sample(range(len(words)), 2)
    words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def random_deletion(text, p=0.2):
    words = text.split()
    if len(words) == 0:
        return text
    new_words = [word for word in words if random.random() > p]
    if len(new_words) == 0:
        return random.choice(words)
    return ' '.join(new_words)

augmented_comments = []
augmented_labels = []

neutral_keywords = ['uncertain', 'not sure', 'confusing', 'doubt', 'unclear']

for idx, row in df.iterrows():
    comment = row['clean_comment']
    label = row['sentiment']

    augmented_comments.append(comment)
    augmented_labels.append(label)

    augmented_comments.append(synonym_replacement(comment))
    augmented_labels.append(label)

    augmented_comments.append(random_insertion(comment))
    augmented_labels.append(label)

    augmented_comments.append(random_swap(comment))
    augmented_labels.append(label)

    augmented_comments.append(random_deletion(comment))
    augmented_labels.append(label)

    if label == 'neutral':
        for kw in neutral_keywords:
            augmented_comments.append(comment + " " + kw)
            augmented_labels.append(label)

aug_df = pd.DataFrame({
    'clean_comment': augmented_comments,
    'sentiment': augmented_labels
})


Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_dat

In [ ]:
aug_df.to_csv("/content/drive/MyDrive/Sentimental_comments/comments_dataset_augmented.csv", index=False)
print("Augmented dataset saved!")

Augmented dataset saved!


In [ ]:
print("Augmented dataset size:", len(aug_df))


Augmented dataset size: 34885


**Split of dataset into train, validate and test**

In [ ]:
sentiment_map = {"negative": 0, "neutral": 1, "positive": 2}
aug_df['label'] = aug_df['sentiment'].map(sentiment_map)

from sklearn.model_selection import train_test_split

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    aug_df['clean_comment'], aug_df['label'], test_size=0.2, random_state=42, stratify=aug_df['label']
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(len(train_texts))
print(len(train_labels))
print(len(val_texts))


27908
27908
3488


**Tokenize the dataset into numbers generating token id**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding="max_length", max_length=90)
val_encodings = tokenizer(list(val_texts), truncation=True, padding="max_length", max_length=90)
test_encodings = tokenizer(list(test_texts), truncation=True, padding="max_length", max_length=90)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

**Wrap up tokenized text data and labels, which will use for training, validation and testing**

In [ ]:
class FeedbackDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

train_dataset = FeedbackDataset(train_encodings, train_labels)
val_dataset = FeedbackDataset(val_encodings, val_labels)
test_dataset = FeedbackDataset(test_encodings, test_labels)


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)  # pos, neg, neutral


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Train the final model with epochs 3**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

OUTPUT_DIR = '/content/drive/MyDrive/Sentimental_comments/MCA21/results'
LOG_DIR = '/content/drive/MyDrive/Sentimental_comments/MCA21/logs'

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=LOG_DIR,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    save_total_limit=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

last_checkpoint = None
if os.path.isdir(OUTPUT_DIR):
    checkpoints = [os.path.join(OUTPUT_DIR, d) for d in os.listdir(OUTPUT_DIR) if 'checkpoint' in d]
    if checkpoints:
        last_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))[-1]
        print(f"Resuming from checkpoint: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)

metrics = trainer.evaluate(test_dataset)
print("Final Test Metrics:", metrics)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.018300,0.012641,0.997133,0.997136,0.997133,0.997133
2,0.007600,0.009733,0.997706,0.997710,0.997706,0.997707
3,0.003900,0.010610,0.997706,0.997706,0.997706,0.997706


Final Test Metrics: {'eval_loss': 0.009390778839588165, 'eval_accuracy': 0.9974204643164231, 'eval_precision': 0.9974241127003965, 'eval_recall': 0.9974204643164231, 'eval_f1': 0.9974211177712253, 'eval_runtime': 8.8541, 'eval_samples_per_second': 394.056, 'eval_steps_per_second': 24.734, 'epoch': 3.0}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = '/content/drive/MyDrive/Sentimental_comments/MCA21/results/checkpoint-5235'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

model.eval()


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/Sentimental_comments/MCA21/results/checkpoint-5235'. Use `repo_type` argument if needed.

In [ ]:
import emoji
import re
import torch

label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

def clean_text(text):
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^a-zA-Z0-9\s!?]', '', text)
    text = text.lower().strip()
    return text

def predict_sentiment(text):
    text = clean_text(text)


    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=90)


    with torch.no_grad():
        outputs = model(**inputs)
        pred_class = torch.argmax(outputs.logits, dim=1).item()


    return label_mapping[pred_class]



In [ ]:
comments = [
    "I really love the new interface, it’s very smooth!",
    "The portal keeps crashing, this is frustrating.",
    "Not sure what to do with these new guidelines.",
    "Wow, the website is down again. What a surprise.",
    ""
]
predictions = [predict_sentiment(c) for c in comments]
print(predictions)


NameError: name 'predict_sentiment' is not defined

**Saving final model to Drive.**

In [ ]:
final_model_dir = "/content/drive/MyDrive/Sentimental_comments/MCA21/final_model"
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model.save_pretrained(final_model_dir)


tokenizer.save_pretrained(final_model_dir)

('/content/drive/MyDrive/Sentimental_comments/MCA21/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/Sentimental_comments/MCA21/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/Sentimental_comments/MCA21/final_model/vocab.txt',
 '/content/drive/MyDrive/Sentimental_comments/MCA21/final_model/added_tokens.json',
 '/content/drive/MyDrive/Sentimental_comments/MCA21/final_model/tokenizer.json')